In [ ]:
import pandas as pd

In [ ]:
import rbot
from rbot import BaseAgent
from rbot import BinanceMarket
from rbot import BackTester
from rbot import time_string

In [ ]:
class Agent(BaseAgent):
    def __init__(self, param_K=1.6):
            self.K = param_K                           # パラメターKを設定する。

    def clock_interval(self):
        #//return 60*60*2
        interval = 60*60*2
        return interval
    
    #def on_tick(self, time_us, session, price, side, size):
    #    print(time_us, price, side, size)

    def on_clock(self, time_us, session):
        ohlcv_df = session.ohlcv(60*60*2, 6)       # ６本の足を取得。 最新は６番目。

        if len(ohlcv_df.index) < 6:                 # データが過去６本分そろっていない場合はなにもせずリターン
            return 

        ohlcv_df["range"] = ohlcv_df["high"] - ohlcv_df["low"]      # レンジを計算

        ohlcv_latest = ohlcv_df[-2:-1]     # 最新足１本
        ohlcv_last_5 = ohlcv_df[:-2]       # 過去５本足

        range_width = ohlcv_last_5["range"].mean()      #　過去５本足のレンジの平均値

        # Long/Short判定
        detect_short = range_width * self.K < ohlcv_latest["high"][0] - ohlcv_latest["open"][0]
        detect_long  = range_width * self.K < ohlcv_latest["open"][0] - ohlcv_latest["low"][0]

        #　執行方法（順報告のポジションがあったら保留。逆方向のポジションがのこっていたらドテン）
        if detect_long:
            #    print("position", session.long_pos_size, session.short_pos_size)            
            #        print("make long")
            if not session.long_position_size:
            #    print("makeorder")                                    
                if not session.short_position_size:

                    session.make_order("Buy", session.buy_board_edge_price, 0.01, 600, "Open Long")    
                else:

                    session.make_order("Buy", session.buy_board_edge_price, 0.02, 600, "doten Long")    
            else:
                pass

        if detect_short:
            #print("make short")            
            if not session.short_position_size:
                #print("makeorder")                                                      
                if not session.long_position_size:
                    session.make_order("Sell", session.sell_board_edge_price, 0.01, 600, "Open Short") 
                else:
                    session.make_order("Sell", session.sell_board_edge_price, 0.02, 600, "Doten Short") 
            else:
                pass

    def on_update(self, time, session, result):
        #print(result)
        pass


In [ ]:
bn = BinanceMarket("BTCBUSD")
bn.download(10)

rbot.init_debug_log()

back_tester = BackTester("BN", "BTCBUSD", False)

print("start")
r = back_tester.run(Agent())
print("end")

print(r)



In [ ]:
import rbot
df = rbot.result_to_df(r)

In [ ]:
df

In [ ]:
df['total_profit'].sum()

In [ ]:
def result_to_df(result_list):
    update_time = []
    order_id = []
    order_sub_id = []
    order_side = []
    post_only = []
    create_time = []
    status = []
    open_price = []
    open_home_size = []
    open_foreign_size = []
    close_price = []
    close_home_size = []
    close_foreign_size = []
    order_price = []
    order_home_size = []
    order_foreign_size = []
    profit = []
    fee = []
    total_profit = []
    message = []

    for item in result_list:
        update_time.append(item.update_time)
        order_id.append(item.order_id)
        order_sub_id.append(item.order_sub_id)
        order_side.append(item.order_side)
        post_only.append(item.post_only)
        create_time.append(item.create_time)
        status.append(item.status)
        open_price.append(item.open_price)
        open_home_size.append(item.open_home_size)
        open_foreign_size.append(item.open_foreign_size)
        close_price.append(item.close_price)
        close_home_size.append(item.close_home_size)
        close_foreign_size.append(item.close_foreign_size)
        order_price.append(item.order_price)
        order_home_size.append(item.order_home_size)
        order_foreign_size.append(item.order_foreign_size)
        profit.append(item.profit)
        fee.append(item.fee)
        total_profit.append(item.total_profit)
        message.append(item.message)

    df = pd.DataFrame(
    data={"update_time": update_time, "order_id": order_id, "sub_id": order_sub_id,
          "order_side": order_side, "post_only": post_only, "create_time": create_time,
          "status":  status, 
          "open_price": open_price, "open_home_size": open_home_size, "open_foreign_size": open_foreign_size, 
          "close_price": close_price, "close_home_size": close_home_size, "close_foreign_size": close_foreign_size,
          "order_price": order_price, "order_home_size": order_home_size, "order_foreign_size": order_foreign_size,
          "profit": profit, "fee": fee,
          "total_profit": total_profit, "message": message},
    columns=["update_time", "order_id", "sub_id", "order_side", "post_only",
             "create_time", "status", "open_price", "open_home_size", "open_foreign_size", 
             "close_price", "close_home_size", "close_foreign_size",             
             "order_price", "order_home_size", "order_foreign_size",
             "profit", "fee", "total_profit", "message"])
    df["update_time"] = pd.to_datetime((df["update_time"]), utc=True, unit="us")
    df["create_time"] = pd.to_datetime((df["create_time"]), utc=True, unit="us")
    df["sum_profit"] = df["total_profit"].cumsum()
    # df["sum_pos"] = df["pos_change"].cumsum()
    df = df.set_index("create_time", drop=True)
    
    return df

In [ ]:
df = result_to_df(r)
df

In [ ]:
bn = BinanceMarket("BTCBUSD")
#bn.download(15)


In [ ]:
bn

In [ ]:
rbot.init_debug_log()
bn.download(10)

In [ ]:
bn.info()

In [ ]:
rbot.init_debug_log()
#print(bn.ohlcvv(1669593599839000 - 60*60*50*1000_000, 1669593599839000, 600))
#print(bn.ohlcvv(1669593599839000 - 60*60*50*1000_000, 1669593599839000, 601))
bn.ohlcvv(0, 1669593599839000, 601)

In [ ]:
bn.ohlcv(0, 0, 600)

In [ ]:
def result_to_df(result_list):
    update_time = []
    order_id = []
    order_sub_id = []
    order_side = []
    post_only = []
    create_time = []
    status = []
    open_price = []
    open_home_size = []
    open_foreign_size = []
    close_price = []
    close_home_size = []
    close_foreign_size = []
    order_price = []
    order_home_size = []
    order_foreign_size = []
    profit = []
    fee = []
    total_profit = []
    message = []

    for item in result_list:
        update_time.append(item.update_time)
        order_id.append(item.order_id)
        order_sub_id.append(item.order_sub_id)
        order_side.append(item.order_side)
        post_only.append(item.post_only)
        create_time.append(item.create_time)
        status.append(item.status)
        open_price.append(item.open_price)
        open_home_size.append(item.open_home_size)
        open_foreign_size.append(item.open_foreign_size)
        close_price.append(item.close_price)
        close_home_size.append(item.close_home_size)
        close_foreign_size.append(item.close_foreign_size)
        order_price.append(item.order_price)
        order_home_size.append(item.home_size)
        order_foreign_size.append(item.order_foreign_size)
        profit.append(item.profit)
        fee.append(item.fee)
        total_profit.append(item.total_profit)
        message.append(item.message)

    df = pd.DataFrame(
    data={"update_time": update_time, "order_id": order_id, "sub_id": order_sub_id,
          "order_side": order_side, "post_only": post_only, "create_time": create_time,
          "status":  status, "open_price": open_price, "open_home_size": open_home_size, 
          "open_foreign_size": open_foreign_size, "close_price": close_price,
          "price": price, "size": size, "volume": volume, "profit": profit, "fee": fee,
          "total_profit": total_profit, "pos_change": position_change, "message": message},
    columns=["update_time", "order_id", "sub_id", "order_type", "post_only",
             "create_time", "status", "open_price", "close_price", "price", "size", "volume",
             "profit", "fee", "total_profit", "pos_change", "message"])
    df["update_time"] = pd.to_datetime((df["update_time"]), utc=True, unit="ms")
    df["create_time"] = pd.to_datetime((df["create_time"]), utc=True, unit="ms")
    df["sum_profit"] = df["total_profit"].cumsum()
    df["sum_pos"] = df["pos_change"].cumsum()
    df = df.set_index("create_time", drop=True)

In [ ]:
result_to_df(r)

In [ ]:
r[0].update_time